In [1]:
from collections import defaultdict
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import LlamaTokenizer, LlamaForCausalLM
from tqdm import tqdm
from peft import PeftModel

seed = 0
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/local_nikhil/.conda/envs/anima/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/home/local_nikhil/Projects/llama_weights/7B"
model = LlamaForCausalLM.from_pretrained(path).to(device)

tokenizer = LlamaTokenizer.from_pretrained(
    "hf-internal-testing/llama-tokenizer", padding_side="right"
)
tokenizer.pad_token_id = tokenizer.eos_token_id

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
raw_data = defaultdict(list)
n_digits = 5
for x in torch.randint(0, pow(10, n_digits), (100,)).tolist():
    for y in torch.randint(0, pow(10, n_digits), (100,)).tolist():
        out = x + y

        x_ = str(x).zfill(n_digits)
        y_ = str(y).zfill(n_digits)
        out = str(out).zfill(n_digits)
        answer = out[0]

        expression = f"x={x_}; y={y_}; x+y="
        raw_data["expression"].append(expression)
        raw_data["label"].append(answer)

In [4]:
# Tokenize dataset['expression'] and store it in input_ids
input_ids = tokenizer(
    raw_data["expression"],
    return_tensors="pt",
)
labels = tokenizer(
    raw_data["label"],
    return_tensors="pt",
)
labels = labels["input_ids"][:, -1]

dataset = Dataset.from_dict(
    {"input_ids": input_ids["input_ids"], "labels": labels}
).with_format("torch")

dataloader = DataLoader(dataset, batch_size=64)

In [5]:
correct, total = 0, 0
for data in tqdm(dataloader, total=len(dataloader)):
    input_ids = data["input_ids"].to(device)
    labels = data["labels"].to(device)

    outputs = model(input_ids=input_ids)
    logits = outputs.logits[:, -1]

    # Get the predicted next token
    predicted_index = torch.argmax(logits, dim=-1)

    # If the prediction is correct, we add 1 to the correct counter
    correct += (predicted_index == labels).sum().item()
    total += labels.size(0)

    del input_ids, labels, outputs, logits, predicted_index
    torch.cuda.empty_cache()

print(f"Accuracy: {round(correct/total, 2)}")

100%|██████████| 157/157 [03:55<00:00,  1.50s/it]

Accuracy: 0.67
